In [67]:
import pandas as pd

In [69]:
data = pd.read_csv("C:/Users/haykn/Downloads/Telegram Desktop/transaction_report.csv")

In [71]:
data.head()

,date,discount_card,store,product_name,card_code,customer_key,customer_address,customer_phone,issue_date,date_of_birth,gender,transaction_amount
0,2024-09-01 08:18:34,NaN,Մասիվ 1,NaN,2717041000767,70501,095 30 21 13,095302113,2010-01-01,NaN,Female,10.0
1,2024-09-01 08:18:34,NaN,Մասիվ 1,NaN,2717041007469,40562,Unknown,+37443305093,2010-01-01,NaN,Female,800.0
2,2024-09-01 08:18:34,NaN,Մասիվ 1,NaN,2717041007469,40562,Unknown,+37443305093,2010-01-01,NaN,Female,30.0
3,2024-09-01 08:18:34,NaN,Մասիվ 1,NaN,2719000246445,12057,Unknown,Unknown,2021-11-05,1985-05-31,Female,320.0
4,2024-09-01 08:18:34,NaN,Մասիվ 1,NaN,2719000106268,50256,նոր նորքի նանսենի 9 բն 29,Unknown,2010-01-01,NaN,Female,920.0


In [82]:
import pandas as pd
from datetime import datetime
from typing import Optional
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

class RFMAnalyzer:
    def __init__(self, data_file) -> None:
        # Load CSV
        self.df = pd.read_csv(data_file, encoding='utf-8-sig')
        if self.df.empty:
            raise ValueError(f"Input file '{data_file}' is empty.")
        if 'card_code' not in self.df.columns:
            raise ValueError("CSV must contain a 'card_code' column for customer IDs.")
        # Preprocess dates and truncate
        self._preprocess_data()
        # Filter to customers: use discount_card if available, else all rows
        if 'discount_card' in self.df.columns and self.df['discount_card'].notna().any():
            self.customer_df = self.df[self.df['discount_card'].notna()].copy()
        else:
            self.customer_df = self.df.copy()
        # Parse truncated date strings back to datetime
        self.customer_df['date'] = pd.to_datetime(self.customer_df['date'], errors='coerce')
        if self.customer_df['date'].isna().all():
            raise ValueError("All 'date' values are NaT after parsing; check your 'date' column format.")
        # Define analysis date as the day after last transaction
        self.analysis_date = self.customer_df['date'].max() + pd.Timedelta(days=1)
        self.rfm: Optional[pd.DataFrame] = None

    def _preprocess_data(self) -> None:
        """Convert date columns to datetime and truncate 'date' to 'YYYY-MM-DD'."""
        for col in ['date', 'issue_date', 'date_of_birth']:
            if col in self.df.columns:
                self.df[col] = pd.to_datetime(self.df[col], errors='coerce')
        self.df['date'] = self.df['date'].dt.strftime('%Y-%m-%d')

    def calculate_rfm(self) -> pd.DataFrame:
        """Compute Recency, Frequency, and Monetary per card_code."""
        self.rfm = (
            self.customer_df
            .groupby('card_code')
            .agg(
                recency   = ('date', lambda x: (self.analysis_date - x.max()).days),
                frequency = ('card_code', 'count'),
                monetary  = ('transaction_amount', 'sum')
            )
            .reset_index()
        )
        return self.rfm

    def score_rfm(self) -> pd.DataFrame:
        """Score R, F, and M on fixed business intervals."""
        if self.rfm is None:
            raise ValueError("Run calculate_rfm() first.")
        def r_score(x): return 5 if x < 5 else 4 if x < 15 else 3 if x < 30 else 2 if x < 60 else 1
        def f_score(x): return 5 if x > 100 else 4 if x > 75 else 3 if x > 40 else 2 if x > 15 else 1
        def m_score(x): return 5 if x > 300_000 else 4 if x > 200_000 else 3 if x > 100_000 else 2 if x > 50_000 else 1

        self.rfm['r_score'] = self.rfm['recency'].apply(r_score)
        self.rfm['f_score'] = self.rfm['frequency'].apply(f_score)
        self.rfm['m_score'] = self.rfm['monetary'].apply(m_score)
        # Create composite codes
        self.rfm['rfm_score'] = (
            self.rfm['r_score'].astype(str)
            + self.rfm['f_score'].astype(str)
            + self.rfm['m_score'].astype(str)
        )
        self.rfm['rfm_sum'] = self.rfm[['r_score', 'f_score', 'm_score']].sum(axis=1, numeric_only=True)
        return self.rfm

    def segment_customers(self) -> pd.DataFrame:
        """Map composite codes to named segments and reclassify unknown codes via KNN."""
        if self.rfm is None:
            raise ValueError("Run score_rfm() first.")
        # Initial regex mapping
        segment_map = {
            r'555|554|545|455|554|544|445|454|544': 'Champions',
            r'4[4-5][4-5]|[4-5][4-5][4-5]|5[4-5][4-5]': 'Loyal Customers',
            r'3[3-5][3-5]|[3-5][3-5][3-5]|4[2-4][2-4]': 'Potential Loyalists',
            r'2[4-5][4-5]|3[2-4][4-5]|4[1-3][4-5]': 'Big Spenders',
            r'[1-2][1-2][1-3]|2[1-2][1-2]|1[2-3][1-2]': 'Leaving Customers'
        }
        # Assign known segments, keep numeric codes for others
        self.rfm['segment'] = (
            self.rfm['rfm_score']
            .replace(segment_map, regex=True)
            .where(lambda s: ~s.str.fullmatch(r'\d{3}'), other=self.rfm['rfm_score'])
        )
        # Merge demographics if available
        demo = []
        for c in ['gender', 'date_of_birth']:
            if c in self.customer_df.columns:
                demo.append(c)
        if demo:
            details = self.customer_df[['card_code'] + demo].drop_duplicates()
            self.rfm = self.rfm.merge(details, on='card_code', how='left')
            if 'date_of_birth' in demo:
                self.rfm['age'] = ((self.analysis_date - pd.to_datetime(self.rfm['date_of_birth']))
                                    .dt.days // 365)
        # Reclassify any numeric-coded segments using KNN
        self.classify_unknown_segments()
        return self.rfm

    def classify_unknown_segments(self, k: int = 5) -> pd.DataFrame:
        """
        Standardize R/F/M scores and use KNN to label any remaining numeric codes.
        Returns rows that were reclassified.
        """
        if self.rfm is None:
            raise ValueError("Run segment_customers() first.")
        mask = self.rfm['segment'].str.fullmatch(r'\d{3}')
        if not mask.any():
            return pd.DataFrame(columns=self.rfm.columns)
        # Prepare data
        known = self.rfm[~mask]
        unknown = self.rfm[mask]
        Xk = known[['r_score','f_score','m_score']]
        yk = known['segment']
        Xu = unknown[['r_score','f_score','m_score']]
        scaler = StandardScaler()
        Xk_s = scaler.fit_transform(Xk)
        Xu_s = scaler.transform(Xu)
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(Xk_s, yk)
        preds = knn.predict(Xu_s)
        self.rfm.loc[mask, 'segment'] = preds
        return self.rfm.loc[mask].copy()

    def analyze_segments(self) -> pd.DataFrame:
        """Aggregate metrics by (now fully-labeled) segment."""
        if self.rfm is None:
            raise ValueError("Run segment_customers() first.")
        result = (
            self.rfm
            .groupby('segment')
            .agg(
                recency   = ('recency','mean'),
                frequency = ('frequency','mean'),
                monetary  = ('monetary','mean'),
                count     = ('card_code','count'),
                age       = ('age','mean')
            )
        )
        result['percentage'] = (result['count']/result['count'].sum())*100
        return result.sort_values('count', ascending=False)

    def save_results(self, filename: str = 'outputs/rfm_results.csv') -> None:
        """Export the full RFM table to CSV."""
        if self.rfm is None:
            raise ValueError("Nothing to save; run your analysis first.")
        self.rfm.to_csv(filename, index=False, encoding='utf-8-sig')


In [84]:
rfm_analyzer = RFMAnalyzer(data_file="C:/Users/haykn/Downloads/Telegram Desktop/transaction_report.csv")
rfm_analyzer.calculate_rfm()
rfm_analyzer.score_rfm()
rfm_analyzer.segment_customers()
segment_stats = rfm_analyzer.analyze_segments()
print(segment_stats)

# Optionally save the output
rfm_analyzer.save_results(filename='C:/Users/haykn/Desktop/rfm_output.csv')


                       recency   frequency       monetary  count        age  \
segment                                                                       
Leaving Customers    71.360188    4.969300   16324.041979   5114  46.700759   
Potential Loyalists   5.797468   25.584005   74936.822768   1738  47.230955   
Champions             1.675676  125.081081  402471.865405     37  48.608696   
Big Spenders         14.333333   13.666667  285065.710000      3  34.000000   
Loyal Customers       5.333333   88.000000  259378.053333      3  36.000000   

                     percentage  
segment                          
Leaving Customers     74.169688  
Potential Loyalists   25.206672  
Champions              0.536621  
Big Spenders           0.043510  
Loyal Customers        0.043510  
